In [7]:
df_csvfile = './workdir/aloi/hmax_df.pickle'

In [8]:
import pickle

In [9]:
df = pickle.load(open( df_csvfile, "rb" ))

In [10]:
df.head()

,category,exemplar,ri_pattern,intact_pattern,ri_typicality,intact_typicality,conserved_typicality
0,798,r225,"[0.4624589128697541, 0.5676057256788559, 0.566...","[0.7798654038796009, 0.8531508057833821, 0.861...",6.997132,6.927355,6.836960
1,77,r225,"[0.4384783079807329, 0.4760773614256729, 0.518...","[0.8060298510002559, 0.857301095365741, 0.8963...",6.999730,6.931086,6.721486
2,925,r180,"[0.4494446489665954, 0.5280466942372747, 0.557...","[0.7437337707542392, 0.8013314013573001, 0.829...",6.999832,6.976707,6.847303
3,62,r90,"[0.415666102437596, 0.5131738643359652, 0.5485...","[0.7498946843798088, 0.8139965558590907, 0.850...",6.999207,6.959101,6.894468
4,291,r90,"[0.4867658386831864, 0.5586230933992825, 0.595...","[0.7297043489848755, 0.7805313261995606, 0.790...",6.999870,6.987722,6.921342
